In [66]:
class MontoContable():
    def __init__(self, tipo, numCta, monto):
        self.tipo = tipo
        self.numCta = numCta
        self.monto = monto
    
    def __str__(self):
        strRes = "(" + self.tipo + ") " + str(self.numCta)
        if self.tipo == 'A':
            strRes += " " * 12
        strRes += " " + str(self.monto)
        return strRes

In [67]:
class PolizaContable ():
    def __init__(self, id, desc, intFecha):
        self.idPoliza = id
        self.descripcion = desc
        self.fecha = intFecha
        self.colMovtos = []

    def cargo(self, idCta, monto):
        m = MontoContable("C", idCta, monto)
        self.colMovtos.append(m)

    def abono(self, idCta, monto):
        m = MontoContable("A", idCta, monto)
        self.colMovtos.append(m)

    def __str__(self):
        strRes = "Póliza " + str(self.idPoliza) + " " + \
            self.descripcion + " " + str(self.fecha) + "\n"
        for m in self.colMovtos:
            strRes += m + "\n"
        return strRes

In [68]:
class CuentaContable():
    def __init__(self, idCta, nombre, nat):
        self.id = idCta
        self.nombre = nombre
        self.nat = nat
        self.colMovtos = []

    def registraMovto(self, movto):
        self.colMovtos.append(movto)
        
    def __str__(self):
        strRes = str(self.id) + " ... " + \
            self.nombre + " ( " + self.nat + " )\n"
        for m in self.colMovtos:
            strRes += str(m) + "\n"
        return strRes


In [74]:
class ParteContable():
    def __init__(self, idParte, nombre, nat):
        self.id = idParte
        self.nombre = nombre
        self.nat = nat
        self.colCtas = {}

    def existeCta(self, idCta):
        return self.colCtas.get(idCta) != None
    
    def altaCta(self, id_cta, nomCta, natCta):
        #print("Parte contable: "+ str(self.id) + 
        #    "Se va a dar de alta la cuentaT: " +
        #      str(id_cta)+ " "+ nomCta + " " + natCta)
        self.colCtas[id_cta] = CuentaContable(id_cta, nomCta, natCta)

    def registraMovto(self, movto):
        self.colCtas[movto.numCta].registraMovto(movto)

    def __str__(self):
        strRes = str(self.id) + " ... " + \
            self.nombre + " ( " + self.nat + " )\n\n"
        for cta in self.colCtas.values():
            strRes+=str(cta) + "\n"
        return strRes

        

In [70]:
class Contabilidad:
    def __init__(self, ejer, nombre):
        self.ejercicio = ejer
        self.nombre = nombre
        self.colPartes = [0] * 6
        self.colPartes[1] = ParteContable(1, "Activo", "D")
        self.colPartes[2] = ParteContable(2, "Pasivo", "A")
        self.colPartes[3] = ParteContable(3, "Capital", "A")
        self.colPartes[4] = ParteContable(4, "Ingresos", "A")
        self.colPartes[5] = ParteContable(5, "Egresos", "D")

    def altaCta(self, id_cta, nomCta, natCta):
        #print("Se vva a dar de alta la cuentaT: " +
        #      str(id_cta)+ " "+ nomCta + " " + natCta)

        # Identifica la parte a la que le corresponde dar de alta la cuenta
        idParte = id_cta//100000
        if 1>idParte or idParte>5:
            raise Exception("La parte " + str(idParte) + "no existe")
        else:
            if self.colPartes[idParte].existeCta(id_cta):
                raise Exception("La cta " + str(id_cta) + "ya exite")
            else:
                return self.colPartes[idParte].altaCta(id_cta, nomCta, natCta)
            
    def registraPoliza(self, pol):
        # verificamos que cuadre
        sc = 0
        sa = 0
        for m in pol.colMovtos:
            if m.tipo == 'C':
                sc += m.monto
            else:
                sa += m.monto
        if sc != sa:
            print("Poliza descuadrada" + str(pol))
            return False
        
        #Verificamos que las cuentas existan
        blnExistenCtas = True
        for m in pol.colMovtos:
            cta = m.numCta
            idParte = cta // 100000
            if 1>idParte or idParte>5 or (not self.colPartes[idParte].existeCta(cta)):
                blnExistenCtas = False
                print("La cta " + str(cta) + " NO EXISTE")
        if not blnExistenCtas:
            return False
        
        #Registramos por cuentas
        for m in pol.colMovtos:
            cta = m.numCta
            idParte = cta // 100000
            self.colPartes[idParte].registraMovto(m)
        return True
    
    def __str__(self):
        strRes = self.nombre + " " + str(self.ejercicio) + "\n"
        for parte in self.colPartes:
            strRes+= str(parte) + "\n\n"
        return strRes

In [75]:
# =============================================================================
# Utilerías Generales
# =============================================================================
import sys
cadsep = "= " * 60
def letrero(strLetrero):
    print(cadsep)
    print(" " + strLetrero)
    print(cadsep)

def salir(strLetrero):
    letrero(strLetrero)
    sys.exit()
# =============================================================================
# Script principal
# =============================================================================
# =============================================================================
# Script principal
# =============================================================================

conta = Contabilidad(2023, "MiEmpre S.A.")

conta.altaCta(100100,"Bancos ","D")
conta.altaCta(100200,"Proveedores ","A")
conta.altaCta(200100,"Capital ","A")
conta.altaCta(300000,"Ventas ","A")
conta.altaCta(400100,"Costo de lo Vendido ","D")
conta.altaCta(500100,"Ni idea ","D")


print(cadsep)
pol1 = PolizaContable(1,"Constitución de la Empresa",20190121)
pol1.cargo(100100,10000)
pol1.abono(300000,10000)
conta.registraPoliza(pol1)
pol2 = PolizaContable(2,"Compra de mercancía por 3000 pagados al contado",20190122)
pol2.cargo(100200,3000)
pol2.abono(100100,3000)
conta.registraPoliza(pol2)
pol3 = PolizaContable(3,"Venta al contado por 1500 de mercancía que costó 1000",20190122)
pol3.abono(100200,1000)
pol3.cargo(500100,1000)
pol3.abono(400100,1500)
pol3.cargo(100100,1500)
conta.registraPoliza(pol3)
pol4 = PolizaContable(4,"Venta al contado por 1000 de mercancía que costó 750",20190201)
pol4.abono(100200, 750)
pol4.cargo(500100, 750)
pol4.abono(400100,1000)
pol4.cargo(500100,1000)
conta.registraPoliza(pol4)
print(cadsep)
#print(conta.balance())
print(cadsep)


#========================== FIN DEL SCRIPT PRINCIPAL =====================

print(str(conta))


= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
= = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = = 
MiEmpre S.A. 2023
0

1 ... Activo ( D )

100100 ... Bancos  ( D )
(C) 100100 10000
(A) 100100             3000
(C) 100100 1500

100200 ... Proveedores  ( A )
(C) 100200 3000
(A) 100200             1000
(A) 100200             750



2 ... Pasivo ( A )

200100 ... Capital  ( A )



3 ... Capital ( A )

300000 ... Ventas  ( A )
(A) 300000             10000



4 ... Ingresos ( A )

400100 ... Costo de lo Vendido  ( D )
(A) 400100             1500
(A) 400100             1000



5 ... Egresos ( D )

500100 ... Ni idea  ( D )
(C) 500100 1000
(C) 500100 750
(C) 500100 1000






In [72]:
print(str(conta.colPartes[1].colCtas[100100]))

100100 ... Bancos  ( D )
(C) 100100 10000
(A) 100100             3000
(C) 100100 1500

